In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np


def evaluate_rmse_forecast(df_forecast, df_real, col_forecast='rate_pred', col_actual='rate'):
    """
    Compute RMSE between forecasted and actual exchange rates (or log_rates).

    Parameters:
    - df_forecast: DataFrame with predicted values (indexed by date)
    - df_real: DataFrame with actual values (indexed by date)
    - col_forecast: column name in df_forecast (e.g. 'rate_pred' or 'log_rate_pred')
    - col_actual: column name in df_real (e.g. 'rate' or 'log_rate')

    Returns:
    - rmse: root mean square error
    """
    # Align both dataframes on index (date)
    df_eval = df_forecast[[col_forecast]].join(df_real[[col_actual]], how='inner')

    # Extract arrays
    y_pred = df_eval[col_forecast].values
    y_true = df_eval[col_actual].values

    # Compute RMSE
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"✅ RMSE between simulated forecast and real data: {rmse:.6f}")
    
    return rmse


In [12]:
# For rates
import pandas as pd
df_test = pd.read_csv('../7. preprocesing/test_df_dexuseu.csv', index_col='date', parse_dates=True)
df_forecast = pd.read_csv('../8. modelling/df_forecast_dexuseu.csv', index_col='date', parse_dates=True)
evaluate_rmse_forecast(df_forecast, df_test, col_forecast='rate_pred', col_actual='rate_interpolated')



✅ RMSE between simulated forecast and real data: 0.188063


0.1880629409794718